# Natural Language Processing 

### PyTorch [Train a word-level language model using Recurrent LSTM networks](https://github.com/pytorch/examples/tree/main/word_language_model)

Natural Language Processing (or NLP for short) is a discipline in computing that deals with the communication between natural (human) languages and computer languages. A common example of NLP is something like spellcheck or autocomplete. Essentially NLP is the field that focuses on how computers can understand and/or process natural/human languages. 

## Recurrent Neural Networks

In this tutorial we will introduce a new kind of neural network that is much more capable of processing sequential data such as text or characters called a **recurrent neural network** (RNN for short). 

We will learn how to use a recurrent neural network to do the following:

- Sentiment Analysis
- Character Generation 

RNN's are complex and come in many different forms so in this tutorial we wil focus on how they work and the kind of problems they are best suited for.

## Sequence Data

In the previous tutorials, we focused on data that we could represent as one static data point where the notion of time or step was irrelevant. Take for example our image data, it was simply a tensor of shape (width, height, channels). That data doesn't change or care about the notion of time. 

In this tutorial we will look at sequences of text and learn how we can encode them in a meaningful way. Unlike images, sequence data such as long chains of text, weather patterns, videos and really anything where the notion of a step or time is relevant needs to be processed and handled in a special way. 

But what do I mean by sequences and why is text data a sequence? Well that's a good question. Since textual data contains many words that follow in a very specific and meaningful order, we need to be able to keep track of each word and when it occurs in the data. Simply encoding say an entire paragraph of text into one data point wouldn't give us a very meaningful picture of the data and would be very difficult to do anything with. This is why we treat text as a sequence and process one word at a time. We will keep track of where each of these words appear and use that information to try to understand the meaning of pieces of text.

## Encoding Text

As we know, machine learning models and neural networks don't take raw text data as an input. This means we must somehow encode our textual data to numeric values that our models can understand. There are many different ways of doing this and we will look at a few examples below. 

Before we get into the different encoding/preprocessing methods let's understand the information we can get from textual data by looking at the following two movie reviews.

```I thought the movie was going to be bad, but it was actually amazing!```

```I thought the movie was going to be amazing, but it was actually bad!```

Although these two sentences are very similar we know that they have very different meanings. This is because of the **ordering** of words, a very important property of textual data.

Now keep that in mind while we consider some different ways of encoding our textual data.

## Bag of Words

The first and simplest way to encode our data is to use something called **bag of words**. This is a pretty easy technique where each word in a sentence is encoded with an integer and thrown into a collection that does not maintain the order of the words but does keep track of the frequency. Have a look at the python function below that encodes a string of text into bag of words. 

In [1]:
vocab = {}  # Map word to integer representing it
word_encoding = 1


def bag_of_words(text):
    global word_encoding

    # Create a list of all the words in the text, we'll assume there is no grammar in our text for this example
    words = text.lower().split(" ")

    # Store all the encodings and their frequency
    bag = {}

    for word in words:
        if word in vocab:
            encoding = vocab[word]  # Get encoding from vocab
        else:
            vocab[word] = word_encoding
            encoding = word_encoding
            word_encoding += 1

        if encoding in bag:
            bag[encoding] += 1
        else:
            bag[encoding] = 1

    return bag


text = "this is a test to see if this test will work is is test a a"
bag = bag_of_words(text)

import pprint as pp

print("\nBag of words:\n")
pp.pprint(bag)

print("\nMap word to integer:\n")
pp.pprint(vocab)


Bag of words:

{1: 2, 2: 3, 3: 3, 4: 3, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1}

Map word to integer:

{'a': 3,
 'if': 7,
 'is': 2,
 'see': 6,
 'test': 4,
 'this': 1,
 'to': 5,
 'will': 8,
 'work': 9}


This isn't the way we would do this in practice, but I hope it gives you an idea of how bag of words works.

Notice that we've lost the order in which words appear.

Let's look at how this encoding works for the two sentences we showed above.

In [2]:
positive_review = "I thought the movie was going to be bad but it was actually amazing"
negative_review = "I thought the movie was going to be amazing but it was actually bad"

pos_bag = bag_of_words(positive_review)
neg_bag = bag_of_words(negative_review)

print("\nPositive:\n", pos_bag)
print("\nNegative:\n", neg_bag)

# They're almost the same.


Positive:
 {10: 1, 11: 1, 12: 1, 13: 1, 14: 2, 15: 1, 5: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1}

Negative:
 {10: 1, 11: 1, 12: 1, 13: 1, 14: 2, 15: 1, 5: 1, 16: 1, 21: 1, 18: 1, 19: 1, 20: 1, 17: 1}


We can see that even though these sentences have a very different meaning they are encoded exactly the same way. 

<span style="color:red;">See if they're the same in Tim's example.</span>

Obviously, this isn't going to fly. Let's look at some other methods.

## Integer Encoding

The next technique we will look at is called **integer encoding**. This involves representing each word or character in a sentence as a unique integer and maintaining the order of these words. This should hopefully fix the problem we saw before were we lost the order of words.

In [3]:
# WE'VE KEPT THE ORDERING IN THIS ONE
vocab = {}
word_encoding = 1


def one_hot_encoding(text):
    global word_encoding

    words = text.lower().split(" ")
    encoding = []  # Instead of putting it in a map, put it in a list this time.

    for word in words:
        if word in vocab:
            code = vocab[word]
            encoding.append(code)
        else:
            vocab[word] = word_encoding
            encoding.append(word_encoding)
            word_encoding += 1

    return encoding


text = "this is a test to see if this test will work is is test a a"
encoding = one_hot_encoding(text)

print("\nOne-Hot Encoding:\n", encoding)
print("\nVocab:\n", vocab)



One-Hot Encoding:
 [1, 2, 3, 4, 5, 6, 7, 1, 4, 8, 9, 2, 2, 4, 3, 3]

Vocab:
 {'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


And now let's have a look at one hot encoding on our movie reviews.

In [4]:
positive_review = "I thought the movie was going to be bad but it was actually amazing"
negative_review = "I thought the movie was going to be amazing but it was actually bad"

pos_encode = one_hot_encoding(positive_review)
neg_encode = one_hot_encoding(negative_review)

print("Positive:", pos_encode)
print("Negative:", neg_encode)

Positive: [10, 11, 12, 13, 14, 15, 5, 16, 17, 18, 19, 14, 20, 21]
Negative: [10, 11, 12, 13, 14, 15, 5, 16, 21, 18, 19, 14, 20, 17]


## Similar words, similar labels

Much better, now we are keeping track of the order of words and we can tell where each occurs. But this still has a few issues with it.

Ideally when we encode words, we would like **similar words to have similar labels** and different words to have very different labels. 

For example, the words **happy** and **joyful** should probably have very similar labels so we can determine that they are similar. While words like horrible and amazing should probably have very different labels. 

The method we looked at above won't be able to do something like this for us. This could mean that the model will have a very difficult time determining if two words are similar or not, which could result in some pretty drastic performance impacts.

## Word Embeddings

Luckily, there is a third method that is far superior, **word embeddings**. This method keeps the order of words intact as well as encodes similar words with very similar labels. It attempts to not only encode the frequency and order of words but the meaning of those words in the sentence. It encodes each word as a dense vector that represents its context in the sentence.

Unlike the previous techniques, word embeddings are learned by looking at many different training examples. You can add what's called an *embedding layer* to the beginning of your model, and while your model trains your embedding layer will learn the correct embeddings for words. You can also use pre-trained embedding layers.

# RNN (Recurrent Neural Network)

Now that we've learned a little bit about how we can encode text, it's time to dive into recurrent neural networks. Up until this point we have been using something called **feed-forward** neural networks. This simply means that all our data is fed forwards (all at once) from left to right through the network. This was fine for the problems we considered before but won't work very well for processing text. After all, even we (humans) don't process text all at once. We read word by word from left to right and keep track of the current meaning of the sentence so we can understand the meaning of the next word. Well this is exactly what a recurrent neural network is designed to do. When we say recurrent neural network, all we really mean is a network that contains a loop. A RNN will process one word at a time while maintaining an internal memory of what it's already seen. This will allow it to treat words differently based on their order in a sentence and to slowly build an understanding of the entire input, one word at a time.

This is why we are treating our text data as a sequence! So that we can pass one word at a time to the RNN.

Let's have a look at what a recurrent layer might look like.

![alt text](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png)
*Source: https://colah.github.io/posts/2015-08-Understanding-LSTMs/*

Let's define what all these variables stand for before we get into the explanation.

**h<sub>t</sub>** output at time t

**x<sub>t</sub>** input at time t

**A** Recurrent Layer (loop)

What this diagram is trying to illustrate is that a recurrent layer processes words or input one at a time in a combination with the output from the previous iteration. So, as we progress further in the input sequence, we build a more complex understanding of the text as a whole.

What we've just looked at is called a <mark>**simple RNN layer.**</mark> It can be effective at processing shorter sequences of text for simple problems but has many downfalls associated with it. One of them being the fact that as text sequences get longer, it gets increasingly difficult for the network to understand the text properly.

# LSTM (Long Short-Term Memory)

The layer we discussed in depth above was called a *simpleRNN*. However, there does exist some other recurrent layers (layers that contain a loop) that work much better than a simple RNN layer. The one we will talk about here is called LSTM (Long Short-Term Memory). This layer works very similarly to the simpleRNN layer but adds a way to access inputs from any time-step in the past. Whereas in our simple RNN layer input from previous timestamps gradually disappeared as we got further through the input. With a LSTM we have a long-term memory data structure storing all the previously seen inputs as well as when we saw them. This allows for us to access any previous value we want at any point in time. This adds to the complexity of our network and allows it to discover more useful relationships between inputs and when they appear.

# Sentiment Analysis

It's time to see a recurrent neural network in action. For this example, we are going to do something called sentiment analysis.

The formal definition of this term from Wikipedia is as follows:

*The process of computationally identifying and categorizing opinions expressed in a piece of text, especially in order to determine whether the writer's attitude towards a particular topic, product, etc. is positive, negative, or neutral.*

The example we'll use here is classifying movie reviews as either positive, negative or neutral.

*This guide is based on the following TensorFlow tutorial: https://www.tensorflow.org/tutorials/text/text_classification_rnn*

## Movie Review Dataset

We'll start by loading the IMDB movie review dataset from Keras. This dataset contains 25,000 reviews from IMDB, where each one is already preprocessed and has a label as either positive or negative. Each review is encoded by integers that represent how common a word is in the entire dataset. For example, a word encoded by the integer 3 means that it is the 3rd most common word in the dataset.

In [5]:
# DATA IS FROM KERAS
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=VOCAB_SIZE)

17464789/17464789 [==============================] - 2s 0us/step


In [6]:
# Lets look at one review
# train_data[1]
len(train_data[1])

189

## More Preprocessing

If we have a look at some of our reviews, we'll notice that they are different lengths. This is an issue. We cannot pass different length data into our neural network. Therefore, we must make each review the same length. To do this we will follow the procedure below:

- If the review is greater than 250 words, trim off the extra words
- If the review is less than 250 words, add the necessary amount of 0's to make it equal to 250.

Keras's **`pad_sequences` function** can do this for us:

In [7]:
from keras.utils import pad_sequences

train_data = keras.utils.pad_sequences(train_data, maxlen=MAXLEN)
test_data = keras.utils.pad_sequences(test_data, maxlen=MAXLEN)

## Creating the Model

Now it's time to create the model. We'll use a word embedding layer as the first layer in our model and add a LSTM layer afterwards that feeds into a dense node to get our predicted sentiment. 

32 stands for the output dimension of the vectors generated by the embedding layer. We can change this value if we'd like!

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),  # word embedding layer
    tf.keras.layers.LSTM(32),  # LSTM layer
    tf.keras.layers.Dense(1, activation="sigmoid")  # dense node => predicted sentiment
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


## Training
Now it's time to compile and train the model. 

In [9]:
# Loss function: binary cross-entropy
# Optimizer: RMSProp (Root Mean Squared Propagation); similar to the gradient descent algorithm with momentum.
# Metrics: accuracy

model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 67s 96ms/step - loss: 0.4596 - acc: 0.7740 - val_loss: 0.3187 - val_acc: 0.8652
Epoch 2/10
625/625 [==============================] - 24s 38ms/step - loss: 0.2623 - acc: 0.8961 - val_loss: 0.2965 - val_acc: 0.8778
Epoch 3/10
625/625 [==============================] - 15s 24ms/step - loss: 0.2036 - acc: 0.9264 - val_loss: 0.2861 - val_acc: 0.8868
Epoch 4/10
625/625 [==============================] - 11s 18ms/step - loss: 0.1622 - acc: 0.9422 - val_loss: 0.4301 - val_acc: 0.8652
Epoch 5/10
625/625 [==============================] - 10s 16ms/step - loss: 0.1365 - acc: 0.9533 - val_loss: 0.2946 - val_acc: 0.8802
Epoch 6/10
625/625 [==============================] - 8s 13ms/step - loss: 0.1171 - acc: 0.9610 - val_loss: 0.3978 - val_acc: 0.8490
Epoch 7/10
625/625 [==============================] - 10s 15ms/step - loss: 0.0950 - acc: 0.9686 - val_loss: 0.3381 - val_acc: 0.8704
Epoch 8/10
625/625 [==============================] - 8s 13ms/s

And we'll evaluate the model on our training data to see how well it performs.

In [10]:
results = model.evaluate(test_data, test_labels)

results

782/782 [==============================] - 4s 6ms/step - loss: 0.5239 - acc: 0.8621


[0.5239357352256775, 0.8621199727058411]

So we're scoring somewhere in the mid-high 80's. Not bad for a simple recurrent network.

## Making Predictions

Let's use our network to make predictions on our own reviews. 

Since our reviews are encoded, we'll need to convert any review that we write into that form, so the network can understand it.

To do that, we'll load the encodings from the dataset, and use them to encode our own data.

In [11]:
word_index = imdb.get_word_index()


def encode_text(text):
    tokens = keras.preprocessing.text.text_to_word_sequence(text)
    print("1", tokens)  # len: 7

    tokens = [word_index[word] if word in word_index else 0 for word in tokens]
    print("2", tokens)

    return keras.utils.pad_sequences([tokens], MAXLEN)[0]


text = "that movie was just amazing, so amazing"
encoded = encode_text(text)

# encoded  # len: 250

1641221/1641221 [==============================] - 1s 1us/step
1 ['that', 'movie', 'was', 'just', 'amazing', 'so', 'amazing']
2 [12, 17, 13, 40, 477, 35, 477]


In [12]:
reverse_word_index = {value: key for (key, value) in word_index.items()}


def decode_integers(integers):
    """
    While we're at it, let's make a decode function.
    """
    PAD = 0
    text = ""
    for num in integers:
        if num != PAD:
            text += reverse_word_index[num] + " "

    return text[:-1]


decode_integers(encoded)

'that movie was just amazing so amazing'

In [13]:
def predict(text):
    """
    Make a prediction
    """
    encoded_text = encode_text(text)
    pred = np.zeros((1, 250))
    pred[0] = encoded_text
    result = model.predict(pred)
    print(result[0])


positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

print()

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)

1 ['that', 'movie', 'was', 'really', 'loved', 'it', 'and', 'would', 'great', 'watch', 'it', 'again', 'because', 'it', 'was', 'amazingly', 'great']
2 [12, 17, 13, 63, 444, 9, 2, 59, 84, 103, 9, 171, 85, 9, 13, 2786, 84]
1/1 [==============================] - 0s 351ms/step
[0.8297435]

1 ['that', 'movie', 'really', 'sucked', 'i', 'hated', 'it', 'and', "wouldn't", 'watch', 'it', 'again', 'was', 'one', 'of', 'the', 'worst', 'things', "i've", 'ever', 'watched']
2 [12, 17, 63, 2064, 10, 1797, 9, 2, 583, 103, 9, 171, 13, 28, 4, 1, 246, 180, 204, 123, 293]
1/1 [==============================] - 0s 19ms/step
[0.11970372]


## Generate a Play with RNN

Time for one of the coolest examples we've seen so far. We are going to use a RNN to generate a play.

We will simply show the RNN an example of something we want it to recreate, and it will learn how to write a version of it on its own. 

We'll do this using a character predictive model that will take as input a variable length sequence and predict the next character. 

We can use the model many times in a row with the output from the last prediction as the input for the next call to generate a sequence.


*Based on: https://www.tensorflow.org/tutorials/text/text_generation*

In [14]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

## Dataset

For this example, we only need one piece of training data. In fact, we can write our own poem or play and pass that to the network for training if we'd like. However, to make things easy we'll use an extract from a Shakespeare play.

In [15]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


## Loading Your Own Data

Upload a file from the dialog below.

Then follow the steps from above, but load in this new file instead.

In [16]:
# from google.colab import files
# path_to_file = list(files.upload().keys())[0]

## Read Contents of File
Let's look at the contents of the file.

In [17]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# Length of text: the number of characters in it
print('\nLength of text: {} characters'.format(len(text)))


Length of text: 1115394 characters


In [18]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



## Encoding

Since this text isn't encoded yet, we'll need to do that ourselves. We are going to encode each unique character as a different integer.

In [19]:
# TODO: Look for a better way.
vocab = sorted(set(text))

# Creating a mapping from unique characters to indices
char2idx = {u: i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)


def text_to_int(text):
    """
    Text to integer
    """
    return np.array([char2idx[c] for c in text])


text_as_int = text_to_int(text)

In [20]:
# Let's look at how part of our text is encoded
print("\nText:", text[:13])
print("\nEncoded:", text_to_int(text[:13]))


Text: First Citizen

Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


Let's make a function that can convert our numeric values to text.

In [21]:
def int_to_text(ints):
    """
    Integer to text
    """
    try:
        ints = ints.numpy()
    except:
        pass
    return ''.join(idx2char[ints])


int_to_text(text_as_int[:13])

'First Citizen'

## Creating Training Examples

Remember our task is to feed the model a sequence and have it return to us the next character. This means we need to split our text data from above into many shorter sequences that we can pass to the model as training examples. 

The training examples we will prepare will use a *seq_length* sequence as input and a *seq_length* sequence as the output where that sequence is the original sequence shifted one letter to the right. For example:

```input: Hell | output: ello```

Our first step will be to create a stream of characters from our text data.

In [22]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text) // (seq_length + 1)  # perform integer division

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

Next we can use the batch method to turn this stream of characters into batches of desired length.

In [23]:
# BATCH
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

Now we need to use these sequences of length 101 and split them into input and output.

In [24]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello


# We use MAP to apply the above function to every entry
dataset = sequences.map(split_input_target)

In [25]:
for x, y in dataset.take(2):
    print("\n\nEXAMPLE\n")
    print("INPUT:")
    print(int_to_text(x))
    print("\nOUTPUT:")
    print(int_to_text(y))



EXAMPLE

INPUT:
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT:
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT:
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT:
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


Finally we need to make training batches.

In [26]:
# MAKE TRAINING BATCHES
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


## Building the Model

Now it is time to build the model. We will use an embedding layer, a LSTM, and one dense layer. The dense layer contains a node for each unique character in our training data. The dense layer will give us a probability distribution over all nodes.

In [27]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    """
    BUILD MODEL
    """
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units,
                             return_sequences=True,
                             stateful=True,
                             recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model


model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (64, None, 256)           16640     
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          5246976   
                                                                 
 dense_1 (Dense)             (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


## Creating a Loss Function

Now we are going to create our own loss function for this problem. This is because our model will output a (64, sequence_length, 65) shaped tensor that represents the probability distribution of each character at each time-step for every sequence in the batch. 

However, before we do that let's have a look at a sample input and the output from our untrained model. So we can understand what the model is giving us.

In [28]:
for input_example_batch, target_example_batch in data.take(1):
    # Ask our model for a prediction on our first batch of training data (64 entries)
    example_batch_predictions = model(input_example_batch)
    
    # Print the output shape
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [29]:
# We can see that the prediction is an array of 64 arrays, one for each entry in the batch

print("\nLen:", len(example_batch_predictions))

# tf.Tensor, shape=(64, 100, 65), dtype=float32)
# print("\nPreds:\n", example_batch_predictions)


Len: 64


In [30]:
# Examine one prediction
pred = example_batch_predictions[0]

print("\nLen:", len(pred))
# print("\nPreds:\n", pred)  # shape=(100, 65)

# Notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step



Len: 100


In [31]:
# Finally, we'll look at a prediction at the first timestep
time_pred = pred[0]

print("\nLen:", len(time_pred))
# print(time_pred)  # tf.Tensor, shape=(65,), dtype=float32)

# And of course it's 65 values, representing the probability of each character occurring next



Len: 65


In [32]:
# If we want to determine the predicted character, we need to sample the output distribution
# (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array...
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]

# ...and convert all the integers to numbers to see the actual characters
predicted_chars = int_to_text(sampled_indices)

# and this is what the model predicted for training sequence 1
predicted_chars


'tCkImGfqVTpo\ngzXKdRV\nUhL,!M\nv;DsKNp?usojNOoRftdgk.B-ugGaPQwb\nZvr Bdw,mPlm.ybHlWPWo;qqzynXsg.nWcee!;J'

So now we need to create a loss function that can compare that output to the expected output and give us some numeric value representing how close the two were.

In [33]:
def loss(labels, logits):
    """
    Custom LOSS FUNCTION
    """
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

## Compiling the Model

At this point, we can think of our problem as a classification problem where the model predicts the probability of each unique letter coming next.

In [34]:
# Adam optimizer, custom loss function
model.compile(optimizer='adam', loss=loss)

## Creating Checkpoints

Setup and configure our model to save checkpoints as it trains. This will allow us to load our model from a checkpoint and continue training it.

In [35]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

# CHECKPOINT CALLBACK
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)


## Training

Finally, we will start training the model. 

**If this is taking a while go to Runtime > Change Runtime Type and choose "GPU" under hardware accelerator.**

In [36]:
# epochs=50
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 16s 72ms/step - loss: 2.6276
Epoch 2/50
172/172 [==============================] - 13s 67ms/step - loss: 1.9134
Epoch 3/50
172/172 [==============================] - 13s 67ms/step - loss: 1.6633
Epoch 4/50
172/172 [==============================] - 13s 67ms/step - loss: 1.5280
Epoch 5/50
172/172 [==============================] - 13s 68ms/step - loss: 1.4458
Epoch 6/50
172/172 [==============================] - 13s 68ms/step - loss: 1.3891
Epoch 7/50
172/172 [==============================] - 13s 68ms/step - loss: 1.3455
Epoch 8/50
172/172 [==============================] - 13s 68ms/step - loss: 1.3097
Epoch 9/50
172/172 [==============================] - 13s 68ms/step - loss: 1.2762
Epoch 10/50
172/172 [==============================] - 14s 70ms/step - loss: 1.2447
Epoch 11/50
172/172 [==============================] - 14s 68ms/step - loss: 1.2149
Epoch 12/50
172/172 [==============================] - 14s 69ms/step - loss: 1.1834
E

## Loading the Model

We'll rebuild the model from a checkpoint using a `batch_size` of 1, so that we can feed one piece of text to the model and have it make a prediction.

In [37]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

### Weights from latest checkpoint

Once the model is finished training, we can find the **latest checkpoint** that stores the models' weights:

In [38]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

### Specify checkpoint

We can load **any checkpoint** we want by specifying the exact file to load.

In [40]:
# TODO: Ensure what ever number you put, you've got!
# checkpoint_num = 10
# model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
# model.build(tf.TensorShape([1, None]))

## Generating Text

Now we can use the lovely function provided by TensorFlow to generate some text using any starting string we'd like.

In [41]:
def generate_text(model, start_string):
    """
    Generate text using the learned model (Evaluation step)
    """
    # Number of characters to generate
    num_generate = 800

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension

        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # We pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)


In [42]:
inp = input("Type a starting string:")

print(generate_text(model, inp))  # She is a bass player


Type a starting string:She is a bass player
She is a bass players;
though you owe to hunt.

First Citizen:
It she so much to them.

CORIOLANUS:
I' the cisto him, as thou art a tower
And the poor foolst inclined starve thus true and supper sent.
Richard your kindness quarrelseto of him living,
Escepting its and sense;
But then i' the mark!--may of bed,
And not impatient like himself;
And now masters, you would say 'tis name with child.

PARIS:
The one now Romeo hath not she
Romeo:--

SICINIUS:
Sir, the sense attaind lord?

LADY CAPULET:
What conn'd by him.'

MENENIUS:
What's the news with you?

POMPEY:
Master Barnardine!

Post:

VOLUMNIA:
That's my courage forbid
What's in the kind of gessen under points,
And blame my humour with flopping them on;
And therefore has are--
More in thy rocky news?

SICINIUS:
We will not flatter me.
I have served Pur seven!


*And* that's pretty much it for this module! I highly recommend messing with the model we just created and seeing what you can get it to do!

## Sources

1. Chollet François. Deep Learning with Python. Manning Publications Co., 2018.
2. "Text Classification with an RNN &nbsp;: &nbsp; TensorFlow Core." TensorFlow, www.tensorflow.org/tutorials/text/text_classification_rnn.
3. "Text Generation with an RNN &nbsp;: &nbsp; TensorFlow Core." TensorFlow, www.tensorflow.org/tutorials/text/text_generation.
4. "Understanding LSTM Networks." Understanding LSTM Networks -- Colah's Blog, https://colah.github.io/posts/2015-08-Understanding-LSTMs/.
5. [TensorFlow with Tim](https://colab.research.google.com/drive/1ysEKrw_LE2jMndo1snrZUh5w87LQsCxk)